# 3. Python

## *Would you be able to replicate the last output (Question 8) above in python pandas library? In that case, could you facilitate us the code?*


Based on the provided material, I have proceeded to obtain the DataFrames "customer_courier_chat_messages" and "Orders" directly from the Excel file. Another valid option, which would prevent formatting errors, would be to obtain these tables from a .csv file using the "pd.read_csv" command. This csv file could be easily obtained from the provided Excel file with minimal manipulation.

**Note.** To simplify the code, the location of the Excel file in the same directory from where we are running the code has been taken into account.

In [1]:
import pandas as pd

# Read from the Excel file
xlsx = pd.ExcelFile('tecnical-test-tables.xlsx')

# Parse the customer_courier_chat_messages table
df_cccm = xlsx.parse(sheet_name="tables", header= 1,skiprows=1, usecols=range(1,10),nrows=12)

# Parse the Orders table
df_ord = xlsx.parse(sheet_name="tables", header= 1,skiprows=16, usecols=range(1,3),nrows=4)

# Inner join between both tables
df = pd.merge(df_cccm, df_ord, how="inner",on='order_id')
col_df_list = list(df.columns.values)

#Visualize the final dataframe
df.head(20)

,sender_app_type,customer_id,from_id,to_id,chat_started_by_message,order_id,order_stage,courier_id,message_sent_time,city_code
0,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:02:05,BCN
1,Courier IOS,99,21,99,False,555,ARRIVING,21,2022-08-09 08:01:15,BCN
2,Customer IOS,99,99,21,False,555,PICKING_UP,21,2022-08-09 08:00:10,BCN
3,Courier Android,122,87,122,True,38,ADDRESS_DELIVERY,87,2022-08-09 07:55:10,BCN
4,Customer Android,43,43,75,False,875,PICKING_UP,75,2022-08-07 14:55:10,VAL
5,Courier Android,43,75,43,False,875,ARRIVING,75,2022-08-07 14:53:10,VAL
6,Customer Android,43,43,75,False,875,PICKING_UP,75,2022-08-07 14:51:10,VAL
7,Courier Android,43,75,43,True,875,ADDRESS_DELIVERY,75,2022-08-07 14:50:10,VAL
8,Customer IOS,23,23,21,False,134,PICKING_UP,21,2022-08-07 10:02:05,OPO
9,Courier IOS,23,21,23,False,134,ARRIVING,21,2022-08-07 10:01:15,OPO


In [2]:
# Validating attributes format
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   sender_app_type          11 non-null     object        
 1   customer_id              11 non-null     int64         
 2   from_id                  11 non-null     int64         
 3   to_id                    11 non-null     int64         
 4   chat_started_by_message  11 non-null     bool          
 5   order_id                 11 non-null     int64         
 6   order_stage              11 non-null     object        
 7   courier_id               11 non-null     int64         
 8   message_sent_time        11 non-null     datetime64[ns]
 9   city_code                11 non-null     object        
dtypes: bool(1), datetime64[ns](1), int64(5), object(3)
memory usage: 891.0+ bytes


The datetime64[ns] format corresponds to "YYYY-mm-dd HH:MM:SS", hence we observe that the date format from Excel has been accurately transferred to the dataframe.
The same applies to the boolean column (TRUE, FALSE) and other columns.

##TASK: Create Dataframe with the following attributes:

*   **order_id**
*   **city_code**
*   **first_courier_message:** Timestamp of the first courier message
*   **first_customer_message:** Timestamp of the first customer message
*   **num_messages_courier:** Number of messages sent by courier
*   **num_messages_customer:**	Number	of	messages	sent	by customer
*   **first_message_by:** The first message sender (courier or customer)
*   **conversation_started_at:** Timestamp of the first message in the conversation
*   **first_responsetime_delay_seconds:** Time (in secs) elapsed until the first message was responded
*   **last_message_time:** Timestamp of the last message sent
*   **last_message_order_stage:** The stage of the order when the last message was sent





In [3]:
#iii.first_courier_message: Timestamp of the first courier message

df_courier_messages = df[df['sender_app_type'].str.contains('Courier')]
first_courier_messages = df_courier_messages.groupby('order_id')['message_sent_time'].min().reset_index()

first_courier_messages = first_courier_messages.rename(columns={'message_sent_time': 'first_courier_message'})
df = pd.merge(df, first_courier_messages, on='order_id', how='left')

In [4]:
#iv.first_customer_message: Timestamp of the first customer message

df_customer_messages = df[df['sender_app_type'].str.contains('Customer')]
first_customer_messages = df_customer_messages.groupby('order_id')['message_sent_time'].min().reset_index()

first_customer_messages = first_customer_messages.rename(columns={'message_sent_time': 'first_customer_message'})
df = pd.merge(df, first_customer_messages, on='order_id', how='left')

In [5]:
#v.num_messages_courier: Number of messages sent by courier

df['num_messages_courier'] = df['sender_app_type'].str.contains('Courier').groupby(df['order_id']).transform('sum')

In [6]:
#vi.num_messages_customer:	Number	of	messages	sent	by customer

df['num_messages_customer'] = df['sender_app_type'].str.contains('Customer').groupby(df['order_id']).transform('sum')

In [7]:
#vii.first_message_by: The first message sender (courier or customer)

## Calculate the minimum message_sent_time for each order_id
min_message_sent_time = df.groupby('order_id')['message_sent_time'].transform('min')

## Get the sender_app_type corresponding to the minimum message_sent_time
df['first_message_by'] = df[df['message_sent_time'] == min_message_sent_time]['sender_app_type']

In [8]:
#viii.conversation_started_at: Timestamp of the first message in the conversation

df['conversation_started_at'] = df.groupby('order_id')['message_sent_time'].transform('min')

In [9]:
#ix.first_responsetime_delay_seconds: Time (in secs) elapsed until the first message was responded

## Sort the DataFrame by 'order_id' and 'message_sent_time'
df = df.sort_values(by=['order_id', 'message_sent_time'])
## Calculate the time difference between the first two messages for each order_id
df['first_responsetime_delay_seconds'] = df.groupby('order_id')['message_sent_time'].transform(lambda x: (x.iloc[1] - x.iloc[0]).total_seconds() if len(x) >= 2 else None)


<ipython-input-9-2c03faf77380>:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['first_responsetime_delay_seconds'] = df.groupby('order_id')['message_sent_time'].transform(lambda x: (x.iloc[1] - x.iloc[0]).total_seconds() if len(x) >= 2 else None)


In [10]:
#x.last_message_time: Timestamp of the last message sent

df['last_message_time'] = df.groupby('order_id')['message_sent_time'].transform('max')

In [11]:
#xi.last_message_order_stage: The stage of the order when the last message was sent

df['last_message_order_stage'] = df.groupby('order_id')['order_stage'].transform('max')


In [ ]:
df_conversation = df.drop_duplicates('order_id')[
    [
    'order_id',
    'city_code',
    'first_courier_message',
    'first_customer_message',
    'num_messages_courier',
    'num_messages_customer',
    'first_message_by',
    'conversation_started_at',
    'first_responsetime_delay_seconds',
    'last_message_time',
    'last_message_order_stage'
    ]]

In [15]:
df_conversation.head()

,order_id,city_code,first_courier_message,first_customer_message,num_messages_courier,num_messages_customer,first_message_by,conversation_started_at,first_responsetime_delay_seconds,last_message_time,last_message_order_stage
3,38,BCN,2022-08-09 07:55:10,NaT,1,0,Courier Android,2022-08-09 07:55:10,NaN,2022-08-09 07:55:10,ADDRESS_DELIVERY
10,134,OPO,2022-08-07 10:01:15,2022-08-07 10:00:10,1,2,Customer IOS,2022-08-07 10:00:10,65.0,2022-08-07 10:02:05,PICKING_UP
2,555,BCN,2022-08-09 08:01:15,2022-08-09 08:00:10,1,2,Customer IOS,2022-08-09 08:00:10,65.0,2022-08-09 08:02:05,PICKING_UP
7,875,VAL,2022-08-07 14:50:10,2022-08-07 14:51:10,2,2,Courier Android,2022-08-07 14:50:10,60.0,2022-08-07 14:55:10,PICKING_UP




---

